# TODO : introduction / paper address / reference / read.md



# Import Libraries

Tensorflow must be imported after scipy

In [31]:
import os
import sys
from datetime import datetime

import numpy as np

import scipy.io
import scipy.misc

import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from PIL import Image
%matplotlib inline

# Input & Output image Setting
<font color='blue'><b> TODO : Edit this to using parser </b></font>

<b>[1]</b> Path for images

<b>[2]</b> Image dimensions <br>

<b>[3]</b> Model - ImageNet VGG-19 <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Normalized model<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(http://bethgelab.org/deepneuralart/)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;MEAN_VALUES is value of each RGB

In [32]:
STYLE_IMAGE = 'images/a.jpg'
CONTENT_IMAGE = 'images/b.jpg'

IMAGE_WIDTH = 436
IMAGE_HEIGHT = 300
COLOR_CHANNELS = 3

VGG_MODEL = 'imagenet-vgg-verydeep-19.mat'
MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

# Hyper Parameter

<b>[1]</b> NOISE_RATIO ($\gamma$) : Constant for initialize <b>white noise image</b> using <b>content image</b>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$Input Image=\gamma*White Noise + (1-\gamma)*Content Image$

<br>
<b>[2]</b> ALPHA & BETA ($\alpha, \beta$) : Ratio of <b>content loss</b> and <b>style loss</b> for <b>total loss</b><br>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathcal{L}_{ total}=\alpha*\mathcal{L}_{content} + \beta*\mathcal{L}_{style}$
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(paper advise $\alpha/\beta$ about $10^{-3}$ ~ $10^{-4}$)

<br>
<b>[3]</b> STYLE_LAYERS & CONTENT_LAYERS : Selected layers from vgg and their weights <br><br>
<b>[4]</b> ITERATIONS : Number of iterations to run<br><br>


In [39]:
NOISE_RATIO = 0.6

ALPHA = 1
BETA = 1000

STYLE_LAYERS = [
    ('conv1_1', 0.3),
    ('conv2_1', 0.25),
    ('conv3_1', 0.2),
    ('conv4_1', 0.15),
    ('conv5_1', 0.1),
]

CONTENT_LAYERS = [
    ('conv4_2', 1.0)
]

ITERATIONS = 5000

def log(file, now):
    file.write('Recorded at: %s\n\n' %now.strftime("%Y-%m-%d_%H:%M:%S"))
    file.write("-----------------------------------------------\n\n")
    file.write('Noise Ratio : %.1f\n\n' %NOISE_RATIO)
    file.write('Alpha : %d\n' %ALPHA)
    file.write('Beta  : %d\n\n' %BETA)
    file.write('Alpha/Beta : %.3f\n\n' %(float(ALPHA)/float(BETA)))
    file.write('Style layers & weights\n')
    for l in STYLE_LAYERS:
        file.write(str(l)+"\n")
    file.write("\n")
    file.write('Content layers & weights\n')
    for l in CONTENT_LAYERS:
        file.write(str(l)+"\n")
    file.write("\n")
    file.write("Iteratioln : %d\n\n" %(ITERATIONS))
    file.write("-----------------------------------------------\n\n")

# Loss Function


$N_{\ell}$ : Number of filters at layer $\ell$  <br>
$M_{\ell}$ : Size of feature map ($width * height$) at layer $\ell$  <br>
$w_{\ell}$ : Weight (contribution) of layer to each loss (style/content)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\sum{w_{\ell}}=1$<br><br>

<b>[1]</b> Content Loss ($\mathcal{L}_{content}$)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$F^{\ell}$ : Activation matrix ($\in \mathcal{R}^{N_\ell \times M_\ell}$) at layer $\ell$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;($F^{\ell}_{ij}$ is the activation of $i^{th}$ filter at position $j$ in layer $\ell$) <br><br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathcal{L}_{content} = \frac{1}{2}\sum_{ij} (F^{\ell}_{{ij}_{output}} - F^{\ell}_{{ij}_{input}})^2$<br><br>

<b>[2]</b> Style Loss ($\mathcal{L}_{style}$)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$G^{\ell}$ : Gram matrix ($\in \mathcal{R}^{N_\ell \times N_\ell}$) - correlation of filters at layer $\ell$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;($G^{\ell}_{ij} = \frac{1}{2}\sum_{k} F^{\ell}_{{ik}}F^{\ell}_{{jk}}$)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$E_{\ell}$ : style loss in layer $\ell$ <br><br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$E_{\ell} = \frac{1}{4N^{2}_{\ell}M^{2}_{\ell}}\sum_{ij} (G^{\ell}_{{ij}_{output}} - G^{\ell}_{{ij}_{input}})^2$<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathcal{L}_{style} = \sum_{\ell} \mathcal{w}_{\ell}E_{\ell}$<br>


In [34]:
def content_loss_func(sess, model):

    def _content_loss(p, x):
        N = p.shape[3]
        M = p.shape[1] * p.shape[2]
        return (1 / 2) * tf.reduce_sum(tf.pow(x - p, 2))
    
    E = [_content_loss(sess.run(model[layer_name]), model[layer_name]) for layer_name, _ in CONTENT_LAYERS]
    W = [w for _, w in CONTENT_LAYERS]
    loss = sum([W[l] * E[l] for l in range(len(CONTENT_LAYERS))])
    return loss

def style_loss_func(sess, model):

    def _gram_matrix(F, N, M):
        Ft = tf.reshape(F, (M, N))
        return tf.matmul(tf.transpose(Ft), Ft)

    def _style_loss(a, x):
        N = a.shape[3]
        M = a.shape[1] * a.shape[2]
    
        A = _gram_matrix(a, N, M)
        G = _gram_matrix(x, N, M)
        result = (1 / (4 * N**2 * M**2)) * tf.reduce_sum(tf.pow(G - A, 2))
        return result

    E = [_style_loss(sess.run(model[layer_name]), model[layer_name]) for layer_name, _ in STYLE_LAYERS]
    W = [w for _, w in STYLE_LAYERS]
    loss = sum([W[l] * E[l] for l in range(len(STYLE_LAYERS))])
    return loss

# Generate/Load/Save Image Util

In [35]:
def generate_noise_image(content_image, noise_ratio = NOISE_RATIO):
    noise_image = np.random.uniform(-20, 20, (1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)).astype('float32')
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    return input_image

def load_image(path):
    image = scipy.misc.imread(path, mode='RGB')
    image = np.reshape(image, ((1,) + image.shape))
    image = image - MEAN_VALUES
    return image

def save_image(path, image):
    image = image + MEAN_VALUES
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

# VGG Model

<b>Notice</b> : paper use average pooling instead of max pooling<br>
<b>Notice</b> : paper use normalized VGG-19 by scaling the weights such that the mean activation of each convolutional filter over images and positions is equal to one. This helps with the image generation and apologise that this was not made clear in the preprint version of the manuscripts.

Load VGG-19 Model and parse it.<br>
W : weight vector<br>
b : bias

In [36]:
def load_vgg_model(path):

    vgg = scipy.io.loadmat(path)

    vgg_layers = vgg['layers']
    
    def _weights(layer, expected_layer_name):

        W = vgg_layers[0][layer][0][0][2][0][0]
        b = vgg_layers[0][layer][0][0][2][0][1]
        layer_name = vgg_layers[0][layer][0][0][0][0]
        
        if expected_layer_name != layer_name:
            print("Warning : layer %s is not matched" % expected_layer_name)
        
        return W, b

    def _relu(conv2d_layer):
        return tf.nn.relu(conv2d_layer)

    def _conv2d(prev_layer, layer, layer_name):
        W, b = _weights(layer, layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.conv2d(
            prev_layer, filter=W, strides=[1, 1, 1, 1], padding='SAME') + b

    def _conv2d_relu(prev_layer, layer, layer_name):
        return _relu(_conv2d(prev_layer, layer, layer_name))

    def _avgpool(prev_layer):
        return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    graph = {}
    graph['input']   = tf.Variable(np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)), dtype = 'float32')
    graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _avgpool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _avgpool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    return graph

# Run



<img src="./images/algorithm.png">

Optimization with ADAM (refer : https://arxiv.org/pdf/1412.6980.pdf) 

In [45]:
def run():
    sess = tf.InteractiveSession()

    content_image = load_image(CONTENT_IMAGE)
    style_image = load_image(STYLE_IMAGE)
    input_image = generate_noise_image(content_image)

    model = load_vgg_model(VGG_MODEL)


    sess.run(tf.global_variables_initializer())

    sess.run(model['input'].assign(content_image))
    content_loss = content_loss_func(sess, model)

    sess.run(model['input'].assign(style_image))
    style_loss = style_loss_func(sess, model)

    total_loss = ALPHA * content_loss + BETA * style_loss


    sess.run(model['input'].assign(input_image))

    optimizer = tf.train.AdamOptimizer(2.0)
    train_step = optimizer.minimize(total_loss)

    sess.run(tf.global_variables_initializer())
    sess.run(model['input'].assign(input_image))

    now = datetime.now()
    case = "result/{:.3f}".format(ALPHA/BETA)+"_"+str(ITERATIONS)+"_"+now.strftime("%Y%m%d_%H%M%S")

    if not os.path.exists("result"):
        os.mkdir("result")

    if not os.path.exists(case):
        os.mkdir(case)

    if not os.path.exists(case+"/output"):
        os.mkdir(case+"/output")

    save_image(case+"/1.content"+CONTENT_IMAGE[CONTENT_IMAGE.rfind('.'):], content_image)
    save_image(case+"/2.style"+STYLE_IMAGE[STYLE_IMAGE.rfind('.'):], style_image)

    file = open(case+"/Log.txt", "a")
    log(file, now)

    for it in range(ITERATIONS+1):
        sess.run(train_step)
        save_it = 1

        if it<10 :
            save_it = 1
        elif it<100 :
            save_it = 5
        elif it<1000 :
            save_it = 10
        else :
            save_it = 100

        if it%save_it == 0:
            mixed_image = sess.run(model['input'])
            file.write('Iteration %d\n' % (it))
            file.write('sum : ' + str(sess.run(tf.reduce_sum(mixed_image))) + "\n")
            file.write('cost: ' + str(sess.run(total_loss)) + "\n\n")

            filename = case+'/output/%04d.png' % (it)
            save_image(filename, mixed_image)

    save_image(case+"/3.result.png", mixed_image)
    file.close()
    sess.close()
    return case

# Optional : Make output pngs to gif

Need images2gif.py<br>
python2 : https://pypi.python.org/pypi/images2gif <br>
python3 : https://github.com/isaacgerg/images2gif <br>

In [49]:
from images2gif import writeGif
from PIL import Image, ImageDraw
import os
import sys
from glob import glob

def makeAnimatedGif(case):

    """
    image files must be sorted

    """
    
    imgFiles = glob(case+"/output/*.png")
    images = [Image.open(fn) for fn in imgFiles]
    
    global filename
    filename = case + "/4.out.gif"
    writeGif(filename, images, duration=0.2)
    print("%s has been created at %s." % (filename, os.path.realpath(filename)))

def start(case):
    print("Creating animated gif....")
    makeAnimatedGif(case)


In [50]:
start("result/0.001_5000_20170218_044855")


Creating animated gif....
result/0.001_5000_20170218_044855/4.out.gif has been created at C:\Users\79981\tensorflow\result\0.001_5000_20170218_044855\4.out.gif.


In [ ]:
STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2),
]


start(run())


ALPHA = 1
BETA = 10000


start(run())


ALPHA = 1
BETA = 100


start(run())


ALPHA = 1
BETA = 1000

STYLE_LAYERS = [
    ('conv1_1', 0.25),
    ('conv2_1', 0.25),
    ('conv3_1', 0.25),
    ('conv4_1', 0.25)
]


start(run())


